# Generate Training Data

We wish to speed up the process of having a human being name entities that they recognize in a paragraph, and having their positions in the paragraph identified and placed in a syntax usable by spaCy's NER training routine.

## Open PDF file, extract page two and display as sentences

In [21]:
import spacy
import PyPDF2
nlp = spacy.load('en_core_web_sm')

#Open PDF file for reading
pdfFile = open("BarCvDescLJ11.pdf", mode="rb")
pdfReader = PyPDF2.PdfFileReader(pdfFile)

# Select a page to work on
pageNumber = 35

# Get text
OnePage = pdfReader.getPage(pageNumber-1) #0-based count
OnePageText = OnePage.extractText()

# Close PDF file
pdfFile.close()

# Remove newlinesxmx82k-&a. It appears multiple newlines together makes
# Spacy think that is the end of a sentence. The PDF reader reads the text in
# an odd fashion
OnePageText = OnePageText.replace('\n','')

# create a spaCy doc object from the page and break it into sentences
doc = nlp(OnePageText)
l=0
for sent in doc.sents:
     print(l, ": ", sent)
     l = l+1


0 :  35  
1 :  UC 937  UC 937 is a six-rowed spring feed barley.
2 :  It was released by the California AES in 1999.
3 :  UC 937 was selected among progeny from the hybrid Sma1/Sunbar 401/3/Gus/Kombyne//Sma1.
4 :  Sma1 has the parentage Steptoe/2*Diamant /3/Minn Dwarf 64.98-8/Briggs/4
5 :  /Asse.
6 :  The final hybridization was made at Woodland, CA in 1983 by Robert Matchett.
7 :  Subsequent generations were handled in a pedigree selection program.
8 :  An F3-derived seed sample in the F6 generation (86Woodland 41233) was received by the University of California, Davis from Robert Matchett in 1987.
9 :  A head row was selected for agronomic appearance in 1992 by Lynn Gallagher and designated UCD 92-10,615 for preliminary yield trials.
10 :  In 1993 it was designated UC 937 and placed in UC regional yield trials for statewide evaluation.
11 :  UC 937 was released primarily for resistance to stripe rust.
12 :  It is intended for late fall to early winter (November-December) sowing in th

## Read in Per-line named entity file and match entities to sentence positions.

In [23]:
import re
import csv
import pandas as pd
fname = "Data/barley_p35_ner.txt"

# Covert the nlp senetence generator into a list of sentences
sentences = list(doc.sents)

# Open the file of manually matched pairs (sentence # <tab> word phrase <tab> named entity)
# e.g.:
#  0      AC Metcalfe     CVAR
#  0      two-rowed       TRAT
#  0      barley          CROP
#  1      Agri-Food Candada   ORG
#  1      1997    DATE
file = open(fname)
reader = csv.reader(file, delimiter='\t', quoting=csv.QUOTE_NONE)
data = list()

for row in reader:
    try:
        (sentIndex, phrase, label) = row
        sent = sentences[int(sentIndex)].string.rstrip()
        
        # find all instances of the 'phrase' in the 'sent'.
        iter = re.finditer(r"\b"+phrase+r"\b", sent)
        indices = [m.start(0) for m in iter]
        
        # check to make sure the phrase the user said was there was indeed found
        if len(indices) == 0:
            raise ValueError
                
        # print out all instances
        for i in indices:
#            print(sentIndex, sent, phrase, "("+str(i), i+len(phrase), "'"+label+"')")
            data.append([sentIndex, sent, phrase, "("+str(i)+", "+str(i+len(phrase))+", '"+label+"')"])
            
    except:
        print("Handle manually: ", row)
        
df = pd.DataFrame(data, columns = ["Index", "Sentence", "Phrase", "MatchInfo"])
print(df)


Handle manually:  ['4', 'Steptoe/2*Diamant /3/Minn Dwarf 64.98-8/Briggs/4/Asse', 'PED']
Handle manually:  ['24', 'Crop Science 42:1374 (2002)', 'JRNL']
Handle manually:  ['37', 'Crop Science 15:603 (1975)', 'JRNL']
Handle manually:  ['52', 'Crop Science 33:1413 (1993)', 'JRNL']
    Index                                           Sentence           Phrase  \
0       1  UC 937  UC 937 is a six-rowed spring feed barley.           UC 937   
1       1  UC 937  UC 937 is a six-rowed spring feed barley.           UC 937   
2       1  UC 937  UC 937 is a six-rowed spring feed barley.        six-rowed   
3       1  UC 937  UC 937 is a six-rowed spring feed barley.           spring   
4       1  UC 937  UC 937 is a six-rowed spring feed barley.             feed   
..    ...                                                ...              ...   
147    69  The crease is closed in the lower one-third an...           crease   
148    69  The crease is closed in the lower one-third an...             

## Create a function to clean up overlapping intervals

In [24]:
import re
coordRegex = re.compile(r'(\d+), (\d+)')

def sortByStart(coords):
    """For use in sort routines, return object with lowest (X,Y) values"""
    # split out coordinates that come in as (5, 7, 'CVAR')
    mo = coordRegex.search(coords)
    return(int(mo.group(1)))

def overlaps(coord1, coord2):
    """Check if coordinates of the form 5, 7, 'CVAR' and 32, 46, 'TRAT' overlap"""
    mo1 = coordRegex.search(coord1)
    mo2 = coordRegex.search(coord2)
    coord1Low = int(mo1.group(1))
    coord1High = int(mo1.group(2))
    coord2Low = int(mo2.group(1))
    coord2High = int(mo2.group(2))
    
    if ((coord1High >= coord2Low) and (coord1Low <= coord2Low) or
        (coord2High >= coord1Low) and (coord2Low <= coord1Low)):
        return True
    else:
        return False

def keepFirst(coord1, coord2):
    """Given overlapping coordinates, return the wider encompassing one."""
    mo1 = coordRegex.search(coord1)
    mo2 = coordRegex.search(coord2)
    coord1Low = int(mo1.group(1))
    coord1High = int(mo1.group(2))
    coord2Low = int(mo2.group(1))
    coord2High = int(mo2.group(2))
 
    if (int(coord1High) - int(coord1Low)) >= (int(coord2High) - int(coord2Low)):
        return True
    else:
        return False

# print("Should be false:", overlaps("(5, 7, 'CVAR')", "(32, 46, 'TRAT')"))
# print("Should be true:", overlaps("(26, 46, 'TRAT')", "(32, 46, 'TRAT')"))
# print("Should be true:", overlaps("(26, 46, 'TRAT')", "(26, 46, 'TRAT')"))
# print("Keeper:", keepFirst("34, 46, 'TRAT'", "34, 46, 'TRAT'"))

def cleanIntervals(inputString=""):
    """order intervals like (5, 7, 'CVAR'), (32, 46, 'TRAT'), (26, 46, 'TRAT') and remove overlapping ones."""
    inputString = inputString.lstrip("(").rstrip(")")
    intervalList = inputString.split("), (")
    intervalList.sort(key = sortByStart)
#    print("Sorted Interval List:", intervalList)

    # Pairwise compare every interval in the list to every other interval to check overlap
    keeperList = [True]*len(intervalList) # Logic array to determine if each interval should be kept
    i=0
    for interval1 in intervalList:
        for interval2 in intervalList:
            if interval1 == interval2:
                if intervalList.index(interval1) != i: # when both are the same we reject the higher one
                    keeperList[i] = False
            else:
                if overlaps(interval1, interval2) and keepFirst(interval1, interval2) == False:
                    keeperList[i] = False
        i = i+1
        
#    print("keeperList:", keeperList)
   
    # Build up the return interval list
    returnStr = "("
    for interval, isKeeper in zip(intervalList, keeperList):
        if isKeeper:
            returnStr = returnStr + interval + "), ("
    return (returnStr.rstrip("), (") + ")")
        
# cleanIntervals("(5, 7, 'CVAR'), (32, 46, 'TRAT'), (5, 9, 'CVAR'), (48, 55, 'ORG'), (26, 46, 'TRAT')")
# cleanIntervals("(0, 8, 'CVAR'), (0, 5, 'CVAR'), (21, 26, 'PLAN'), (32, 37, 'CVAR')")
cleanIntervals("(0, 12, 'CVAR'), (39, 49, 'CVAR'), (39, 49, 'CVAR'), (71, 77, 'CVAR'), (71, 77, 'CVAR'), (92, 113, 'TRAT'), (140, 150, 'CVAR'), (140, 150, 'CVAR'), (181, 187, 'CVAR'), (181, 187, 'CVAR')")


"(0, 12, 'CVAR'), (39, 49, 'CVAR'), (71, 77, 'CVAR'), (92, 113, 'TRAT'), (140, 150, 'CVAR'), (181, 187, 'CVAR')"

## Aggregate all matches for each sentence on a single line and output in spaCy training format

In [25]:
# use Pandas dataframes to aggregate all entity matches together for a single sentence
agg_rules = {'Sentence': 'first', 'Phrase': 'first', 'MatchInfo': lambda x: ', '.join(x)}
res = df.groupby('Index').agg(agg_rules)
#print(res)

# Now format it just like what is needed for the spaCy training module: 
# E.g.:
# ('Eight-Twelve is a six-rowed winter feed barley', {'entities': [(0, 12, 'CVAR'), (18, 27, 'TRAT'), (28, 39, 'TRAT'),(40, 46, 'CROP')]}),
records = res.to_dict('records')
for i in range(0,len(records)):
    print("    ('"+records[i]['Sentence']+"', {'entities': ["+cleanIntervals(records[i]['MatchInfo'])+"]}),")
    #print("    ('"+records[i]['Sentence']+"', {'entities': ["+records[i]['MatchInfo']+"]}),")


    ('UC 937  UC 937 is a six-rowed spring feed barley.', {'entities': [(0, 6, 'CVAR'), (8, 14, 'CVAR'), (20, 29, 'TRAT'), (30, 36, 'TRAT'), (37, 41, 'TRAT'), (42, 48, 'CROP')]}),
    ('UC 937 was released primarily for resistance to stripe rust.', {'entities': [(0, 6, 'CVAR'), (34, 44, 'PPTD'), (48, 59, 'PATH')]}),
    ('UC 937 is a medium tall semi-dwarf (moderately short-statured as a result of the sdw gene, but averages two inches taller than UC 828) with fair straw strength.', {'entities': [(0, 6, 'CVAR'), (19, 23, 'TRAT'), (24, 34, 'TRAT'), (47, 61, 'TRAT'), (115, 121, 'TRAT'), (127, 133, 'CVAR'), (145, 159, 'TRAT')]}),
    ('It is medium-late maturing', {'entities': [(18, 26, 'TRAT')]}),
    ('(heading time is 2 to 5 days later than UC 828 with mid-November to mid-December emergence under short daylength in California).', {'entities': [(1, 13, 'TRAT'), (40, 46, 'CVAR'), (81, 90, 'TRAT')]}),
    ('Spikes are fully awned, semi-erect, mid-dense and waxy.', {'entities': [(0, 6, 'PLA